# HUMAN ACTIVITY RECOGNITION

In [1]:
#importing necessry libararies
import pandas as pd
import numpy as np
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
import keras
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(23)
import tensorflow as tf
tf.set_random_seed(23)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [12]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [13]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [14]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


# Architecture of LSTM WITH ONE LAYER

WITH 32 LSTM UNITS

In [15]:
epochs = 30
batch_size = 16
n_hidden = 32
import keras

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,kernel_initializer='glorot_uniform',kernel_regularizer=keras.regularizers.l2(0.01),input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes,activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 54s 7ms/step - loss: 1.3438 - acc: 0.4415 - val_loss: 1.2514 - val_acc: 0.5110
Epoch 2/30
7352/7352 [==============================] - 45s 6ms/step - loss: 1.1137 - acc: 0.5260 - val_loss: 1.1821 - val_acc: 0.5195
Epoch 3/30
7352/7352 [==============================] - 44s 6ms/step - loss: 0.9173 - acc: 0.6646 - val_loss: 0.8440 - val_acc: 0.7194
Epoch 4/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.7092 - acc: 0.7582 - val_loss: 0.7426 - val_acc: 0.7462
Epoch 5/30
7352/7352 [==============================] - 38s 5ms/step - loss: 0.6015 - acc: 0.8071 - val_loss: 0.6821 - val_acc: 0.7937
Epoch 6/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.5019 - acc: 0.8496 - val_loss: 0.5336 - val_acc: 0.8351
Epoch 7/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.4343 - acc: 0.8

In [19]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      372       112        2                   0   
STANDING                 0       86       444        2                   0   
WALKING                  3        0         0      483                   0   
WALKING_DOWNSTAIRS       0        0         0       10                 395   
WALKING_UPSTAIRS         7        6         1       51                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           10  
WALKING_DOWNSTAIRS                15  
WALKING_UPSTAIRS                 406  


In [20]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 607us/step


In [21]:
score

[0.3903591258076538, 0.8948082796063793]

WITH 64 LSTM UNITS

In [22]:
epochs = 30
batch_size = 16
n_hidden = 64

In [16]:
#n_steps, n_length = 4, 32
#X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, input_dim))
#X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, input_dim))

In [23]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 57s 8ms/step - loss: 1.1235 - acc: 0.5223 - val_loss: 1.0438 - val_acc: 0.5636
Epoch 2/30
7352/7352 [==============================] - 55s 7ms/step - loss: 0.7175 - acc: 0.7055 - val_loss: 1.4433 - val_acc: 0.6013
Epoch 3/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.5717 - acc: 0.8044 - val_loss: 0.5565 - val_acc: 0.8334
Epoch 4/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.3381 - acc: 0.8908 - val_loss: 0.3908 - val_acc: 0.8622
Epoch 5/30
7352/7352 [==============================] - 55s 7ms/step - loss: 0.2398 - acc: 0.9192 - val_loss: 0.4973 - val_acc: 0.8717
Epoch 6/30
7352/7352 [==============================] - 54s 7ms/step - loss: 0.2284 - acc: 0.9275 - val_loss: 0.5707 - val_acc: 0.8558
Epoch 7/30
7352/7352 [==============================] - 54s 7ms/step - loss: 0.2063 - acc: 0.9378 - val_loss: 0.4014 - val_acc: 0.8694
Epoch 8

In [26]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        25        0                   0   
SITTING                  0      388       100        0                   0   
STANDING                 0      110       421        1                   0   
WALKING                  0        0         0      455                  14   
WALKING_DOWNSTAIRS       0        0         1        0                 412   
WALKING_UPSTAIRS         0        1         3        5                  13   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            3  
STANDING                           0  
WALKING                           27  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 449  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 921us/step


In [28]:
score

[0.46545276161816074, 0.8941296233457754]

WITH 128 LSTM UNITS

In [29]:
epochs = 30
batch_size = 16
n_hidden = 128

In [30]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 90s 12ms/step - loss: 1.1401 - acc: 0.5316 - val_loss: 0.8731 - val_acc: 0.6284
Epoch 2/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.6759 - acc: 0.7282 - val_loss: 0.6944 - val_acc: 0.7733
Epoch 3/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.4599 - acc: 0.8429 - val_loss: 0.4666 - val_acc: 0.8588
Epoch 4/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.2931 - acc: 0.9070 - val_loss: 0.3649 - val_acc: 0.8700
Epoch 5/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.3860 - acc: 0.8692 - val_loss: 0.5013 - val_acc: 0.8154
Epoch 6/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.4117 - acc: 0.8690 - val_loss: 0.4298 - val_acc: 0.8578
Epoch 7/30
7352/7352 [==============================] - 88s 12ms/step - loss: 0.2912 - acc: 0.9047 - val_loss: 0.3491 - val_acc: 0.8724


In [33]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 511        0        13       13                   0   
SITTING                  0      353       137        0                   0   
STANDING                 0       41       489        2                   0   
WALKING                  0        0         0      464                   8   
WALKING_DOWNSTAIRS       0        0         0       12                 397   
WALKING_UPSTAIRS         0        1         2       33                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           24  
WALKING_DOWNSTAIRS                11  
WALKING_UPSTAIRS                 435  


In [34]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [35]:
score

[0.3158544494973829, 0.8988802171700034]

with Dropout 0.4

In [36]:
epochs = 30
batch_size = 256
n_hidden = 128

In [37]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [39]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 44s 6ms/step - loss: 1.4212 - acc: 0.3715 - val_loss: 1.3190 - val_acc: 0.3482
Epoch 2/30
7352/7352 [==============================] - 44s 6ms/step - loss: 1.3187 - acc: 0.4188 - val_loss: 1.2344 - val_acc: 0.4469
Epoch 3/30
7352/7352 [==============================] - 46s 6ms/step - loss: 1.1797 - acc: 0.4795 - val_loss: 1.2065 - val_acc: 0.4601
Epoch 4/30
7352/7352 [==============================] - 49s 7ms/step - loss: 1.1811 - acc: 0.4853 - val_loss: 1.2274 - val_acc: 0.4659
Epoch 5/30
7352/7352 [==============================] - 50s 7ms/step - loss: 1.1110 - acc: 0.5185 - val_loss: 1.2072 - val_acc: 0.4859
Epoch 6/30
7352/7352 [==============================] - 52s 7ms/step - loss: 1.0865 - acc: 0.5284 - val_loss: 1.1120 - val_acc: 0.5684
Epoch 7/30
7352/7352 [==============================] - 52s 7ms/step - loss: 1.0423 - acc: 0.5396 - val_loss: 1.0371 - val_acc: 0.5436
Epoch 8

In [40]:
# Confusion Matrix
print(confusion_matrix(Y_test,model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  2      408        81        0                   0   
STANDING                 0      119       411        0                   0   
WALKING                  0        1         0      448                  28   
WALKING_DOWNSTAIRS       0        0         0        3                 403   
WALKING_UPSTAIRS         0        0         2       29                  32   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           2  
WALKING                           19  
WALKING_DOWNSTAIRS                14  
WALKING_UPSTAIRS                 408  


In [41]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 2ms/step


In [42]:
score

[0.3383634945306286, 0.8781812012215813]

# Architecture of LSTM WITH TWO LAYERS

In [52]:
epochs = 30
batch_size = 16
n_hidden1 = 32
n_hidden2 =20

In [54]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden1,kernel_initializer='glorot_uniform', return_sequences=True,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.4))
model.add(LSTM(n_hidden2,kernel_initializer='glorot_uniform'))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20)                4240      
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 126       
Total params: 9,742
Trainable params: 9,742
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [56]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 91s 12ms/step - loss: 1.2679 - acc: 0.4437 - val_loss: 1.1566 - val_acc: 0.4774
Epoch 2/30
7352/7352 [==============================] - 89s 12ms/step - loss: 0.9006 - acc: 0.6366 - val_loss: 0.8959 - val_acc: 0.6715
Epoch 3/30
7352/7352 [==============================] - 97s 13ms/step - loss: 0.6396 - acc: 0.7518 - val_loss: 0.7133 - val_acc: 0.7194
Epoch 4/30
7352/7352 [==============================] - 92s 12ms/step - loss: 0.5221 - acc: 0.7748 - val_loss: 0.6854 - val_acc: 0.7262
Epoch 5/30
7352/7352 [==============================] - 93s 13ms/step - loss: 0.4422 - acc: 0.8089 - val_loss: 0.6795 - val_acc: 0.7458
Epoch 6/30
7352/7352 [==============================] - 91s 12ms/step - loss: 0.3255 - acc: 0.8780 - val_loss: 0.5078 - val_acc: 0.8351
Epoch 7/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.2210 - acc: 0.9251 - val_loss: 0.4889 - val_acc: 0.8497


In [57]:
# Confusion Matrix
print(confusion_matrix(Y_test,model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      415        53        4                   0   
STANDING                 0       69       462        1                   0   
WALKING                  0        0         0      481                  11   
WALKING_DOWNSTAIRS       0        0         0        1                 415   
WALKING_UPSTAIRS         0        0         0       25                   7   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           19  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 439  


In [58]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 1ms/step


In [59]:
score

[0.33934266308311606, 0.9236511706820495]

CNN+LSTM

In [88]:
epochs = 30
kernel_size = 3 #kernel_size of 1 worked surprisingly well
pool_size = 2
dropout_rate = 0.15
f_act = 'relu'

In [89]:
from keras.layers import BatchNormalization

In [94]:
#https://blog.goodaudience.com/predicting-physical-activity-based-on-smartphone-sensor-data-using-cnn-lstm-9182dd13b6bc
first_model = Sequential()
first_model.add(Conv1D(512, (kernel_size), input_shape=(X_train.shape[1],X_train.shape[2]), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(Dropout(dropout_rate))
first_model.add(Conv1D(64, (kernel_size), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(Dropout(dropout_rate))
first_model.add(Conv1D(32, (kernel_size), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(LSTM(128, return_sequences=True))
first_model.add(LSTM(128, return_sequences=True))
first_model.add(LSTM(128))
first_model.add(Dropout(dropout_rate))
first_model.add(Dense(n_classes, activation='softmax'))

In [95]:
first_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [96]:
filepath="best_model_weights2.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
callbacks_list =[checkpoint]

In [97]:
history=first_model.fit(X_train,
          Y_train,
          batch_size=batch_size,callbacks=callbacks_list,
          validation_data=(X_test, Y_test),
          epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 189s 26ms/step - loss: 0.8546 - acc: 0.7296 - val_loss: 0.6212 - val_acc: 0.7913

Epoch 00001: val_acc improved from -inf to 0.79131, saving model to best_model_weights2.hdf5
Epoch 2/30
7352/7352 [==============================] - 187s 25ms/step - loss: 0.3340 - acc: 0.9029 - val_loss: 0.2809 - val_acc: 0.8996

Epoch 00002: val_acc improved from 0.79131 to 0.89956, saving model to best_model_weights2.hdf5
Epoch 3/30
7352/7352 [==============================] - 188s 26ms/step - loss: 0.1964 - acc: 0.9354 - val_loss: 0.3422 - val_acc: 0.8707

Epoch 00003: val_acc did not improve from 0.89956
Epoch 4/30
7352/7352 [==============================] - 187s 25ms/step - loss: 0.1614 - acc: 0.9427 - val_loss: 0.2332 - val_acc: 0.9060

Epoch 00004: val_acc improved from 0.89956 to 0.90601, saving model to best_model_weights2.hdf5
Epoch 5/30
7352/7352 [==============================] - 186s 25ms

LOADING THE PREVIOUSLY STORED MODEL

In [98]:
#https://blog.goodaudience.com/predicting-physical-activity-based-on-smartphone-sensor-data-using-cnn-lstm-9182dd13b6bc
first_model = Sequential()
first_model.add(Conv1D(512, (kernel_size), input_shape=(X_train.shape[1],X_train.shape[2]), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(Dropout(dropout_rate))
first_model.add(Conv1D(64, (kernel_size), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(Dropout(dropout_rate))
first_model.add(Conv1D(32, (kernel_size), activation=f_act, padding='same'))
first_model.add(BatchNormalization())
first_model.add(MaxPooling1D(pool_size=(pool_size),padding='same'))
first_model.add(LSTM(128, return_sequences=True))
first_model.add(LSTM(128, return_sequences=True))
first_model.add(LSTM(128))
first_model.add(Dropout(dropout_rate))
first_model.add(Dense(n_classes, activation='softmax'))

In [99]:
first_model.load_weights("best_model_weights2.hdf5")#loading the weights from saved file

In [100]:
first_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [101]:
print(confusion_matrix(Y_test,first_model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      418        64        0                   0   
STANDING                 0       62       470        0                   0   
WALKING                  0        0         0      496                   0   
WALKING_DOWNSTAIRS       0        0         0       12                 406   
WALKING_UPSTAIRS         0        0         0        1                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            7  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 469  


In [102]:
score = first_model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 22s 7ms/step


In [103]:
score

[0.16700983426927404, 0.9487614523243977]

In [104]:
from prettytable import PrettyTable
Z=PrettyTable()
Z.field_names = ["Model","Test_loss","Test_Accuracy"]
Z.add_row(["LSTM(32)","0.3903591258076538", "0.8948082796063793"])
Z.add_row(["LSTM(64)","0.46545276161816074","0.8941296233457754"])
Z.add_row(["LSTM(128)","0.3158544494973829","0.8988802171700034"])
Z.add_row(["LSTM(128)-Dropout(0.4)","0.3383634945306286", "0.8781812012215813"])
Z.add_row(["LSTM(32)-LSTM(20)","0.3393426630831160","0.9236511706820495"])
Z.add_row(["CONV1D-LSTM","0.16700983426927404","0.9487614523243977"])


print(Z)

+------------------------+---------------------+--------------------+
|         Model          |      Test_loss      |   Test_Accuracy    |
+------------------------+---------------------+--------------------+
|        LSTM(32)        |  0.3903591258076538 | 0.8948082796063793 |
|        LSTM(64)        | 0.46545276161816074 | 0.8941296233457754 |
|       LSTM(128)        |  0.3158544494973829 | 0.8988802171700034 |
| LSTM(128)-Dropout(0.4) |  0.3383634945306286 | 0.8781812012215813 |
|   LSTM(32)-LSTM(20)    |  0.3393426630831160 | 0.9236511706820495 |
|      CONV1D-LSTM       | 0.16700983426927404 | 0.9487614523243977 |
+------------------------+---------------------+--------------------+


# CONCLUSION

1. First we tried with single lstm layer to improve the accuracy.

2. Because of small number of data points if we use more number of lstms model is very easiliy leads to overfitting.
    
3. Even After Experimenting with various number of lstms and dropouts also we are not able to improve accuracy  significantly.

4. After That we used 2 layer lstm to tackle the problem.But not successful in improving accuaracy.

5. we used conv1d with lstm to improve model's performance.

6. we used 3 convolutional Layers and 3 lstm layers in model's architecture.

7. And used The keras callback feature To save the best model in all the epochs.

8. Best Model is with the accuracy 94.88%